# ProRail Storing Analyse & Voorspellingen
Voor het project Data-driven Business hebben wij de opdracht gekregen om ProRail te helpen bij het voorspellen van de hersteltijd van een storing. We hebben van ProRail een dataset gekregen waar alles in staat over storingen in het verleden. Nu is het aan ons om de data op de juiste manier te analyseren en er voorspellende modellen bij te maken.

## Analyse
Het is erg belangrijk om de data grondig te analysereren. We hebben op dit moment nog geen idee waar we mee werken, dus gaan we dat uitzoeken.

### Inladen & Configureren
Voor dat we aan de analyse gaan beginnen moeten de benodigdheden worden geïmporteerd en geconfigureerd worden. Pandas en Numpy zijn ervoor om de data te analyseren. SciKit-Learn is er voor om de modellen te maken, en Joblib gebruiken we aan het einde om de modellen op te slaan. Daarnaast zijn er nog een aantal configuraties die gedaan moeten worden om het proces soepeler te laten verlopen.

#### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import joblib

#### Configuratie

In [2]:
pd.set_option('display.max_columns', None)

in de data zijn values in een column met verschillende Type, met low_memory=False kunt pandas bij inlezen een faste Type voor een column vast leggen.

In [3]:
df = pd.read_csv('sap_storing_data_hu_project.csv', index_col=0, low_memory=False)
df.head()

,#stm_sap_meldnr,stm_mon_nr,stm_vl_post,stm_sap_meld_ddt,stm_sap_mon_meld_ddt,stm_sap_meldtekst_lang,stm_mon_begin_ddt,stm_sap_meldtekst,stm_mon_toelichting_trdl,stm_oh_pg_mld,stm_geo_mld,stm_geo_mld_uit_functiepl,stm_equipm_nr_mld,stm_equipm_soort_mld,stm_equipm_omschr_mld,stm_km_van_mld,stm_km_tot_mld,stm_prioriteit,stm_scenario_mon,stm_status_melding_sap,stm_mon_nr_status_omschr,stm_mon_nr__statuscode,stm_mon_nr_status_wijzdd,stm_aanngeb_ddt,stm_aanntpl_ddt,stm_oh_pg_gst,stm_geo_gst,stm_geo_gst_uit_functiepl,stm_equipm_nr_gst,stm_equipm_soort_gst,stm_equipm_omschr_gst,stm_objectdl_code_gst,stm_objectdl_groep_gst,stm_km_van_gst,stm_km_tot_gst,stm_progfh_in_ddt,stm_progfh_in_invoer_ddt,stm_progfh_gw_ddt,stm_progfh_gw_lwd_ddt,stm_progfh_hz,stm_oorz_groep,stm_oorz_code,stm_oorz_tkst,stm_veroorz_groep,stm_veroorz_code,stm_veroorz_tekst_kort,stm_effect,stm_afspr_aanvangddt,stm_fh_ddt,stm_fh_status,stm_sap_storeind_ddt,stm_mon_eind_ddt,stm_mon_vhdsincident,stm_tao_indicator,stm_tao_indicator_vorige,stm_tao_soort_mutatie,stm_tao_telling_mutatie,stm_tao_beinvloedbaar_indicator,stm_evb,stm_dir_betrok_tr,stm_aangelegd_dd,stm_aangelegd_tijd,stm_sap_melddatum,stm_sap_meldtijd,stm_mon_begindatum,stm_mon_begintijd,stm_contractgeb_mld,stm_functiepl_mld,stm_techn_mld,stm_contractgeb_gst,stm_functiepl_gst,stm_techn_gst,stm_aanngeb_dd,stm_aanngeb_tijd,stm_aanntpl_dd,stm_aanntpl_tijd,stm_arbeid,stm_progfh_in_datum,stm_progfh_in_tijd,stm_progfh_in_invoer_dat,stm_progfh_in_invoer_tijd,stm_progfh_in_duur,stm_progfh_gw_datum,stm_progfh_gw_tijd,stm_progfh_gw_lwd_datum,stm_progfh_gw_lwd_tijd,stm_progfh_gw_duur,stm_progfh_gw_teller,stm_afspr_aanvangdd,stm_afspr_aanvangtijd,stm_fh_dd,stm_fh_tijd,stm_fh_duur,stm_reactie_duur,stm_sap_storeinddatum,stm_sap_storeindtijd,stm_mon_eind_datum,stm_mon_eind_tijd,stm_controle_dd,stm_akkoord_mon_toewijz,stm_status_sapnaarmon,stm_fact_jn,stm_akkoord_melding_jn,stm_afsluit_ddt,stm_afsluit_dd,stm_afsluit_tijd,stm_rec_toegev_ddt,stm_hinderwaarde,stm_actie,stm_standplaats,stm_status_gebr,stm_wbi_nummer,stm_projnr,stm_oorz_tekst_kort,stm_historie_toelichting,stm_schade_verhaalb_jn,stm_schadenr,stm_schade_status_ga,stm_schade_statusdatum,stm_relatiervo_vorig,stm_relatiervo_volgend,stm_relatiervo,stm_pplg_van,stm_pplg_naar,stm_dstrglp_van,stm_dstrglp_naar,stm_afspr_func_hersteldd,stm_afspr_func_hersteltijd,stm_sorteerveld,stm_rapportage_maand,stm_rapportage_jaar,stm_x_bron_publ_dt,stm_x_bron_bestandsnaam,stm_x_bron_arch_dt,stm_x_actueel_ind,stm_x_run_id,stm_x_bk,stm_x_start_sessie_dt,stm_x_vervallen_ind
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,0,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2014,07/05/2014 05:30:12,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,31/12/9999 00:00:00,1,34415,3617016,07/05/2014 05:44:24,0
1,50053211,0.0,NaN,02/01/2006 09:00:00,02/01/2006 09:00:00,Logboeknr Geeltje : 49 Tijd: 0900 VL-Po...,NaN,Logboeknr Geeltje : 49 Tijd: 0900,NaN,NaN,624.0,624.0,NaN,NaN,NaN,0.0,0.0,9.0,NaN,MAFS MAFD,NaN,NaN,00/00/0000,02/01/2006 09:00:00,NaN,81S,624.0,624.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,Z,ONR-RIB,133.0,Papieren ontbreken,ONREGRIB,B,Volker,NaN,NaN,02/01/2006 09:00:00,4.0,02/01/2006 09:00:00,NaN,NaN,N,NaN,NaN,0,B,"0,00",NaN,03/01/2006,04:48:18,02/01/2006,09:00:00,NaN,NaN,NaN,624,NaN,81.0,624,S,02/01/2006,09:00:00,NaN,00:00:00,99999999.0,NaN,00:00:00,NaN,00:00:00,99999999.0,NaN,00:00:00,NaN,NaN,99999999.0,0.0,00/00/0000,00:00:00,02/01/2006,09:00:00,0.0,99999999.0,02/01/2006,09:00:00,NaN,NaN,05/01/2006,J,NaN,N,J,05/01/2009 15:10:09,05/01/2009,15:10:09,02/01/2006 04:48:18,0.0,NaN,N

Er zijn een aantal kolommen waar helemaal geen waardes in staan. Die gaan we er dus uit filteren, aangeien we er niks aan hebben.

In [4]:
df = df[[
    "stm_mon_nr", "stm_vl_post", "stm_sap_meld_ddt", "stm_sap_meldtekst_lang",
    "stm_sap_meldtekst", "stm_geo_mld", "stm_equipm_nr_mld", "stm_equipm_soort_mld",
    "stm_equipm_omschr_mld", "stm_km_van_mld", "stm_km_tot_mld", "stm_prioriteit",
    "stm_aanngeb_ddt", "stm_oh_pg_gst", "stm_geo_gst", "stm_equipm_nr_gst",
    "stm_equipm_soort_gst", "stm_equipm_omschr_gst", "stm_km_van_gst", "stm_km_tot_gst",
    "stm_oorz_groep", "stm_oorz_code", "stm_oorz_tkst", "stm_fh_ddt",
    "stm_fh_status", "stm_sap_storeind_ddt", "stm_tao_indicator", "stm_tao_indicator_vorige",
    "stm_tao_soort_mutatie", "stm_tao_telling_mutatie", "stm_tao_beinvloedbaar_indicator", "stm_sap_melddatum",
    "stm_sap_meldtijd", "stm_contractgeb_mld", "stm_techn_mld", "stm_contractgeb_gst",
    "stm_techn_gst", "stm_aanngeb_dd", "stm_aanngeb_tijd", "stm_aanntpl_dd",
    "stm_aanntpl_tijd", "stm_progfh_in_datum", "stm_progfh_in_tijd", "stm_progfh_in_invoer_dat",
    "stm_progfh_in_invoer_tijd", "stm_progfh_in_duur", "stm_progfh_gw_tijd", "stm_progfh_gw_teller",
    "stm_fh_dd", "stm_fh_tijd", "stm_fh_duur", "stm_sap_storeinddatum",
    "stm_sap_storeindtijd", "stm_oorz_tekst_kort", "stm_pplg_van", "stm_pplg_naar",
    "stm_dstrglp_van", "stm_dstrglp_naar"
]]

#### Opvallend

De data bestaat voornamelijk uit drie soorten gegevens:
- Tekstuele inhoud
- Numerieke klassen
- Datum en tijd

Tijdens het proces wanneer het model wordt gebruikt, zijn niet alle data beschikbaar. Kolommen die eindigen met _gst zijn niet geschikt.

Ook gaan we kijken of er dubbele rijen zijn die er uit gefilterd kunnen worden.

In [5]:
df = df.drop_duplicates()

#### Bruikbare Features

Omdat de dataset enorm en vrij ingewikkeld is. Hebben we samen met de opdrachtgever gezeten om te bepalen welke kolommen relevant zijn, zodat we de rest er uit kunnen filteren.

In [6]:
data = df[[
    'stm_oorz_code', 'stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_geo_mld',
    'stm_aanntpl_tijd', 'stm_fh_tijd', 'stm_techn_mld', 'stm_prioriteit',
    'stm_contractgeb_mld', 'stm_fh_duur', 'stm_progfh_in_duur', 'stm_progfh_in_tijd'
]]

Hier zijn de betekenissen en meetwaardes van de kolommen die we gaan gebruiken:
| Kolom | Betekenis | Meetwaarde |
|-|-|-|
| stm_oorz_code | Oorzaak code | Nominaal |
| stm_sap_melddatum | Datum melding | Ordinaal |
| stm_sap_meldtijd | Tijdstip melding | Ordinaal |
| stm_geo_mld | Geo code melding | Nominaal |
| stm_aanntpl_tijd | Tijdstip aannemer ter plaatse | Ordinaal |
| stm_fh_tijd | Tijdstip functieherstel | Ordinaal |
| stm_techn_mld | Techniekveld melding | Nominaal |
| stm_prioriteit | Prioriteitsindicatie | Ordinaal |
| stm_contractgeb_mld | Contract gebied melding | Nominaal |
| stm_fh_duur | Duur functieherstel | Continue |
| stm_progfh_in_duur | Prognose duur functieherstel | Continue |
| stm_progfh_in_tijd | Prognose tijd functieherstel | Ordinaal |

#### Converteren Tijden

We gaan hier de tijden converteren van HH:MM:SS formaat naar minuten sinds middernacht, zodat we er berekeningen mee kunnen uitvoeren.

In [7]:
# Kolomnamen met tijd in HH:MM:SS formaat
tijd_kolommen = ['stm_sap_meldtijd', 'stm_aanntpl_tijd', 'stm_fh_tijd','stm_progfh_in_tijd']

# Functie om tijdstring om te zetten naar totaal aantal minuten
def convert_time_to_minutes(df, columns):
    for col in columns:
        df.loc[:, col] = pd.to_datetime(df[col], format='%H:%M:%S', errors='coerce')
        df.loc[:, col] = df[col].apply(lambda x: x.hour * 60 + x.minute if pd.notnull(x) else None)
    return df


#### Functie Hersteltijd

We willen de functie hersteltijd gaan voorspellen, dus zullen we moeten kijken hoe deze kolom zich verhoudt to de andere kolommen in de data.

In [8]:
data = convert_time_to_minutes(data, tijd_kolommen)

In [9]:
data[['stm_sap_meldtijd', 'stm_aanntpl_tijd', 'stm_fh_tijd','stm_fh_duur']]

,stm_sap_meldtijd,stm_aanntpl_tijd,stm_fh_tijd,stm_fh_duur
0,NaN,NaN,NaN,NaN
1,540.0,0.0,540.0,0.0
2,755.0,0.0,806.0,51.0
3,1000.0,0.0,1040.0,40.0
4,1350.0,0.0,1356.0,6.0
...,...,...,...,...
908625,486.0,545.0,569.0,83.0
908626,561.0,608.0,644.0,83.0
908627,561.0,608.0,644.0,83.0
908628,855.0,885.0,914.0,19.0


Hieruit blijkt dat functie hersteel duur gelijk aan hersteel tijdstip - meldings tijdstip<br>
ook zien je bij somig storing dat geen infomatie is wanneer aannemer aanweizig was.

In [10]:
len(data[data['stm_fh_duur'] == 0])

160611

In [11]:
len(data.loc[(data["stm_progfh_in_tijd"] == data["stm_fh_tijd"]) & (df["stm_fh_tijd"] != 0)])

291345

In [12]:
len(data[data['stm_fh_duur'] == (data['stm_progfh_in_tijd']-data['stm_aanntpl_tijd'])])

156596

Hieruit kunnen we zien dat 160.000 data geen stroning bij melding.<br>
140.000 data prognose functie herstel tijdstip gelijk zijn aan werkelijk hersteltijd<br>
150.00 data zijn werkelijk FHT gelijk aan tijdstip aannemer te plaats tot prognose functie herstel tijdstip 

in stm_progfh_in_duur zijn str values, we bewaar eerste alle numeric waardes en zet we de type als int

In [13]:
data.loc[:, 'stm_progfh_in_duur'] = pd.to_numeric(data['stm_progfh_in_duur'], errors='coerce')
data.loc[:, 'stm_progfh_in_duur'] = data['stm_progfh_in_duur'].fillna(0).astype(int)

In [14]:
len(data[data['stm_progfh_in_duur'] == (data['stm_progfh_in_tijd']-data['stm_aanntpl_tijd'])])

486618

In [15]:
len(data[data['stm_progfh_in_duur'] == data['stm_fh_duur']])

95215

verder is te zien dat, 120.000 data dat prognose functie herstel tijd gelijk tijdstip aannemer te plaats + prognose functie herstel tijdstip <br>
ook zijn 9.000 data dat prognose duur gelijk aan werkelijk FHT duur

# Target

voor de traget neem we de tijd neem de functie hersteltijd van waarneer de aanmener aanweizig is tot dat weer gereiden mogen worden.<br>
we bewaar eerst alle data dat binnen dag is

In [16]:
data['targetherstel'] = np.where(
    data['stm_fh_tijd'] - data['stm_aanntpl_tijd']>= 0, # als de stm_fh_tijd-stm_aanntpl_tijd positief is
    data['stm_fh_tijd'] - data['stm_aanntpl_tijd'], # dan betekende dat nog op de zefde day is, kun je gelijk van elkaar aftrekken
    data['stm_fh_tijd'] - data['stm_aanntpl_tijd']+1440 # als niet op zelfdde dag is hersteld, dus een neagetief uitkomst moet er +1440 om de juiste tijd te krijgen.
)

C:\Users\maste\AppData\Local\Temp\ipykernel_26736\3034723497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['targetherstel'] = np.where(


In [17]:
ranges = {
    "null": (0, 0),
    "tot 15m": (0, 15),
    "15m tot 30m": (15, 30),
    "30m tot 1h": (30, 60),
    "1h tot 2h": (60, 120),
    "2h tot 3h": (120, 180),
    "3h tot 6h": (180, 360),
    "6h tot 8h": (360, 480),
    "8h+": (540, float('inf'))
}

total = len(data)

for label, (low, high) in ranges.items():
    if label == "null":
        count = len(data[data['targetherstel'] == 0])
    else:
        count = len(data[(data['targetherstel'] > low) & (data['targetherstel'] <= high)])
    print(f'score {label}: {count/total:.2%}')


score null: 20.53%
score tot 15m: 21.31%
score 15m tot 30m: 12.51%
score 30m tot 1h: 15.28%
score 1h tot 2h: 12.22%
score 2h tot 3h: 4.81%
score 3h tot 6h: 5.22%
score 6h tot 8h: 1.20%
score 8h+: 6.38%


we zien dat dat 20% van de meldingen geen probleem zijn dat de spoor op stil staat
21% dat onder de 15 miuten is en 13% tussen 15-30 min
voor meldingen dat de spoor boven de acht uurr stil staan zij er maar 6%

In [18]:
len(data[data['stm_progfh_in_duur'] == data['targetherstel']])

196142

# Filter

filter data's dat stm_fh_duur onder de 15 min en data's boven de 8 uur,het is niet belangrjik in deze project.<br>
we nemen data's van 15min tot 8uur, zo data de model niet telaag gaat onderschaten.

In [19]:
data = data.loc[ (data['targetherstel']>15) & (data['targetherstel']<=480)]

Filter van grognose duur, er mag geen rare waardes als 0 of 99999999 in staan

In [20]:
data = data.loc[(data['stm_progfh_in_duur']>0) & (data['stm_progfh_in_duur']<=1440)]

Filter voor menselijk fouten waar de waarders achteraf is door de aannemer ingevoerd zijn.

In [21]:
data = data.loc[(data["stm_progfh_in_tijd"] != data["stm_fh_tijd"])]

In [22]:
data = data[data['stm_progfh_in_duur'] != data['stm_fh_duur']]

In [23]:
data = data[data['stm_progfh_in_duur'] != data['targetherstel']]

# datum reparatie

In [24]:
data = data.copy()
#Zet de kolom om naar datetime,vervang de datum door de dag van het jaar
data.loc[:, 'stm_sap_melddatum'] = pd.to_datetime(data['stm_sap_melddatum'], format='%d/%m/%Y').dt.dayofyear

# prioriteit

In [25]:
data=data.loc[(data['stm_prioriteit']!=8)&(data['stm_prioriteit']!=9)]

In [26]:
data

,stm_oorz_code,stm_sap_melddatum,stm_sap_meldtijd,stm_geo_mld,stm_aanntpl_tijd,stm_fh_tijd,stm_techn_mld,stm_prioriteit,stm_contractgeb_mld,stm_fh_duur,stm_progfh_in_duur,stm_progfh_in_tijd,targetherstel
98626,215.0,201,545.0,119.0,30.0,240.0,NaN,2.0,NaN,1012.0,270,300.0,210.0
98627,215.0,201,545.0,119.0,30.0,240.0,NaN,2.0,NaN,1012.0,270,300.0,210.0
144452,215.0,279,749.0,63.0,788.0,810.0,S,4.0,26.0,53.0,31,819.0,22.0
144453,215.0,279,749.0,63.0,788.0,810.0,S,4.0,26.0,53.0,31,819.0,22.0
144457,215.0,279,1012.0,62.0,583.0,632.0,S,4.0,25.0,1051.0,41,624.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
908625,215.0,131,486.0,560.0,545.0,569.0,NaN,5.0,NaN,83.0,30,575.0,24.0
908626,218.0,131,561.0,468.0,608.0,644.0,S,2.0,4.0,83.0,52,660.0,36.0
908627,218.0,131,561.0,468.0,608.0,644.0,S,2.0,4.0,83.0,52,660.0,36.0
908628,135.0,233,855.0,102.0,885.0,914.0,NaN,2.0,NaN,19.0,75,960.0,29.0


# Retypen

In [27]:
le = LabelEncoder()

data['stm_techn_mld_encoded'] = le.fit_transform(data['stm_techn_mld'])
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [28]:
print(label_mapping)

{'A': 0, 'B': 1, 'E': 2, 'G': 3, 'I': 4, 'K': 5, 'M': 6, 'O': 7, 'P': 8, 'S': 9, 'T': 10, 'X': 11, nan: 12}


In [29]:
data = data.fillna(0)

In [30]:
int_columns = ['stm_oorz_code','stm_sap_melddatum','stm_sap_meldtijd','stm_geo_mld','stm_aanntpl_tijd','stm_fh_tijd','stm_progfh_in_duur',
    'stm_progfh_in_tijd','targetherstel','stm_contractgeb_mld','stm_prioriteit','stm_fh_duur','stm_techn_mld_encoded']

for col in int_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce').astype('Int64')

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240974 entries, 98626 to 908629
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   stm_oorz_code          240974 non-null  Int64 
 1   stm_sap_melddatum      240974 non-null  Int64 
 2   stm_sap_meldtijd       240974 non-null  Int64 
 3   stm_geo_mld            240974 non-null  Int64 
 4   stm_aanntpl_tijd       240974 non-null  Int64 
 5   stm_fh_tijd            240974 non-null  Int64 
 6   stm_techn_mld          240974 non-null  object
 7   stm_prioriteit         240974 non-null  Int64 
 8   stm_contractgeb_mld    240974 non-null  Int64 
 9   stm_fh_duur            240974 non-null  Int64 
 10  stm_progfh_in_duur     240974 non-null  Int64 
 11  stm_progfh_in_tijd     240974 non-null  Int64 
 12  targetherstel          240974 non-null  Int64 
 13  stm_techn_mld_encoded  240974 non-null  Int64 
dtypes: Int64(13), object(1)
memory usage: 30.6+ MB


## Modellen

### Baseline

We neem RMSE van prognose tijd duur met de traget als de baseline voor de Linearrregressie model.

In [32]:
np.sqrt(mean_squared_error(data['stm_progfh_in_duur'], data['targetherstel']))

197.8312425579125

Naast de RMSE waarde, wanneer kan we zeggen of een verspelling ook een goede verspelling is?<br> Als de verspelling target herstel tijd kan bedekken met een maximale verschil van 15 minuten.

In [33]:
tolerance = 15

correct = (data['stm_progfh_in_duur'] > data['targetherstel']) & \
          (abs(data['stm_progfh_in_duur'] - data['targetherstel']) <= tolerance)

accuracy = np.mean(correct)

print(f"Accuracy (overschatting binnen {tolerance} minuten): {accuracy:.2%}")

Accuracy (overschatting binnen 15 minuten): 35.31%


In [34]:
np.abs(data['stm_progfh_in_duur'] - data['targetherstel']) 

98626     60
98627     60
144452     9
144453     9
144457     8
          ..
908625     6
908626    16
908627    16
908628    46
908629    46
Length: 240974, dtype: Int64

In [35]:
len(data[data['stm_progfh_in_duur'] > data['targetherstel']])/len(data)

0.8246781810485778

Uit data blijk 35.24% van data van de prognose zal goede verspelling zijn, waarbij 82% van alle data altijd de targetherstel tijd kunt bedekken is.<br>
Het is belangrijk dat de voorspelling beter overschat dan onderschat in de project.

Veder is te zien, als model altijd 30 minuten voorspeelt, heeft het model een accuracy van 35,15% heeft.

In [36]:
ranges = {
    "15m tot 30m": (15, 30),
    "30m tot 45m": (30, 45),
}

total = len(data)

for label, (low, high) in ranges.items():
    if label == "null":
        count = len(data[data['targetherstel'] == 0])
    else:
        count = len(data[(data['targetherstel'] > low) & (data['targetherstel'] <= high)])
    print(f'score {label}: {count/total:.2%}')


score 15m tot 30m: 24.82%
score 30m tot 45m: 17.52%


Dus wanner is het model beter dan nu?<br>
1.Als RMSE kleiner wordt<br>
2.Als verspelling met marge van 15 minuten het Target kunt bedekken en een hogere scoren boven 42% accuracy heeft<br>
3.Als de totale verspelling een betere dekkingsgraad heeft.

### Linear Regression

In [37]:
def Train_per_categorical(data):
    data = data.copy()  # Voorkom SettingWithCopyWarning

    # Categorische kolommen die één voor één toegevoegd worden
    categorical_cols = ['stm_oorz_code', 'stm_geo_mld', 'stm_contractgeb_mld', 'stm_techn_mld', 'stm_prioriteit']

    # Numerieke kolommen die altijd in het model zitten
    fixed_numeric_cols = ['stm_sap_meldtijd','stm_sap_melddatum','stm_aanntpl_tijd', 'stm_progfh_in_duur']

    # Eerst de baseline (alleen vaste numerieke features)
    print("Baseline model: alleen numerieke features")
    X_base = data[fixed_numeric_cols]
    y = data['targetherstel']

    X_train, X_test, y_train, y_test = train_test_split(X_base, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"[Baseline] R² = {r2:.3f}, RMSE = {rmse:.2f}")

    # Accuracy en coverage
    tolerance = 15
    correct = (data['stm_progfh_in_duur'] > data['targetherstel']) & \
              ((data['stm_progfh_in_duur'] - data['targetherstel']) <= tolerance)
    accuracy = np.mean(correct)
    coverage = np.mean(data['stm_progfh_in_duur'] > data['targetherstel'])
    print(f"[Baseline] Accuracy (±{tolerance} min overschatting): {accuracy:.2%}")
    print(f"[Baseline] coverage: {coverage:.2%}")

    # Vervolgens per categorische kolom apart toevoegen
    for col in categorical_cols:
        print(f"\nModel trainen met categorische kolom: '{col}' + vaste numerieke features")

        data[col] = data[col].astype(str)
        X_cat = pd.get_dummies(data[[col]], drop_first=True)

        X = pd.concat([data[fixed_numeric_cols], X_cat], axis=1)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

        print(f"R² = {r2:.3f}, RMSE = {rmse:.2f}")

        # Accuracy & coverage opnieuw berekenen (zelfde formule)
        correct = (y_pred > y_test) & \
                  (abs(y_pred - y_test) <= 15)
        accuracy = np.mean(correct)
        coverage = np.mean(data['stm_progfh_in_duur'] > data['targetherstel'])

        print(f"Accuracy (±{tolerance} min overschatting): {accuracy:.2%}")
        print(f"coverage: {coverage:.2%}")



In [38]:
Train_per_categorical(data)

Baseline model: alleen numerieke features
[Baseline] R² = 0.124, RMSE = 72.05
[Baseline] Accuracy (±15 min overschatting): 35.31%
[Baseline] coverage: 82.47%

Model trainen met categorische kolom: 'stm_oorz_code' + vaste numerieke features
R² = 0.164, RMSE = 70.39
Accuracy (±15 min overschatting): 13.45%
coverage: 82.47%

Model trainen met categorische kolom: 'stm_geo_mld' + vaste numerieke features
R² = 0.130, RMSE = 71.80
Accuracy (±15 min overschatting): 10.86%
coverage: 82.47%

Model trainen met categorische kolom: 'stm_contractgeb_mld' + vaste numerieke features
R² = 0.130, RMSE = 71.82
Accuracy (±15 min overschatting): 10.78%
coverage: 82.47%

Model trainen met categorische kolom: 'stm_techn_mld' + vaste numerieke features
R² = 0.132, RMSE = 71.71
Accuracy (±15 min overschatting): 11.10%
coverage: 82.47%

Model trainen met categorische kolom: 'stm_prioriteit' + vaste numerieke features
R² = 0.126, RMSE = 71.96
Accuracy (±15 min overschatting): 10.90%
coverage: 82.47%


## conclusie

Naast de datum, meldings tijd, aannemer anweizig tijd en prognse functie herstel duur, is per categorische kolom gekeken of er bijdraag is voor de model. Maar leidt niet op beter uitkomst komt.<br>
<br>
Vergelijken met de baseline heeft het model een lager RMSE waarde. Maar de rest is ongeveer zelfde gebleven. Verder is het accuracy ook niet hooger dan 42.15%, dus het model presteert niet echt beter dan nu

## save model

In [39]:
X = data[['stm_sap_meldtijd','stm_sap_melddatum','stm_aanntpl_tijd', 'stm_progfh_in_duur']]
y = data['targetherstel']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [42]:
joblib.dump(model, 'LinearRegressionModel.joblib')

['LinearRegressionModel.joblib']

## Decision Tree

Voor de decision tree is gekozen voor het trainen van model om de target in verschillen de classen te verdelen <br>
gatllen tusen 15 tot 30 min wordt klase 1<br>
30 to 45 min wordt klase 2<br>
....

In [43]:
def build_klassen(start=15, end=481, step=15):
    klassen = {}
    index = 1
    for lower in range(start, end, step):
        upper = lower + step
        klassen[index] = (lower, upper)
        index += 1
    return klassen

klassen = build_klassen()

In [44]:
def categorize_herstel(minuten):
    for label, (min_val, max_val) in klassen.items():
        if min_val <= minuten < max_val:
            return label
    return 0
data['herstel_klasse'] = data['targetherstel'].apply(categorize_herstel)

In [45]:
data

,stm_oorz_code,stm_sap_melddatum,stm_sap_meldtijd,stm_geo_mld,stm_aanntpl_tijd,stm_fh_tijd,stm_techn_mld,stm_prioriteit,stm_contractgeb_mld,stm_fh_duur,stm_progfh_in_duur,stm_progfh_in_tijd,targetherstel,stm_techn_mld_encoded,herstel_klasse
98626,215,201,545,119,30,240,0,2,0,1012,270,300,210,12,14
98627,215,201,545,119,30,240,0,2,0,1012,270,300,210,12,14
144452,215,279,749,63,788,810,S,4,26,53,31,819,22,9,1
144453,215,279,749,63,788,810,S,4,26,53,31,819,22,9,1
144457,215,279,1012,62,583,632,S,4,25,1051,41,624,49,9,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908625,215,131,486,560,545,569,0,5,0,83,30,575,24,12,1
908626,218,131,561,468,608,644,S,2,4,83,52,660,36,9,2
908627,218,131,561,468,608,644,S,2,4,83,52,660,36,9,2
908628,135,233,855,102,885,914,0,2,0,19,75,960,29,12,1


In [48]:
def train_decision_tree(data):
    """Train verbeterd Decision Tree model met hyperparameter tuning"""
    data = data.copy()

    # Alle feature categorieën
    numeric_cols = ['stm_progfh_in_duur', 'stm_sap_melddatum', 'stm_sap_meldtijd',
                   'stm_aanntpl_tijd', 'stm_prioriteit']

    # Feature engineering: voeg afgeleide features toe
    data['tijd_verschil_meld_aanntpl'] = data['stm_aanntpl_tijd'] - data['stm_sap_meldtijd']

    # Voeg tijdsgerelateerde features toe
    data['is_werkdag'] = (data['stm_sap_melddatum'] % 7 < 5).astype(int)
    data['uur_van_dag'] = (data['stm_sap_meldtijd'] // 60).astype(int)
    data['is_spitsuur'] = ((data['uur_van_dag'] >= 7) & (data['uur_van_dag'] <= 9) |
                          (data['uur_van_dag'] >= 17) & (data['uur_van_dag'] <= 19)).astype(int)

    # Update numeric columns met nieuwe features
    numeric_cols.extend(['tijd_verschil_meld_aanntpl', 'is_werkdag',
                        'uur_van_dag', 'is_spitsuur','stm_oorz_code', 'stm_geo_mld', 'stm_contractgeb_mld','stm_techn_mld_encoded'])

    # Prepare features
    X_numeric = data[numeric_cols]
    


    # One-hot encode categorische features met frequentie filtering
    X_categorical = pd.DataFrame()

    # Combineer alle features
    X = pd.concat([X_numeric, X_categorical], axis=1)
    y = data[['targetherstel', 'herstel_klasse']]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    y_train = y_train['herstel_klasse']
    y_test = y_test['targetherstel']
    

    # Verbeterd Decision Tree model met betere hyperparameters
    model = DecisionTreeRegressor(
        random_state=42,
        max_depth=20,           # Dieper voor meer complexiteit
        min_samples_split=20,   # Hoger voor regularisatie
        min_samples_leaf=10,    # Hoger voor regularisatie
        max_features=0.7,       # Meer features beschikbaar
        min_impurity_decrease=0.001,  # Verminder overfitting
        ccp_alpha=0.01          # Cost complexity pruning
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # predict keer 15 om weer in minuten te krijgen
    y_pred_minutes = y_pred * 15

    # Evaluatie
    r2 = r2_score(y_pred_minutes,y_test)
    rmse = np.sqrt(mean_squared_error(y_pred_minutes,y_test))

    print("Verbeterd Decision Tree model:")
    print(f"R² = {r2:.3f}, RMSE = {rmse:.2f}")
    print(f"Aantal features: {X.shape[1]}")

    tolerance =15
    correct = (y_pred_minutes +14 >= y_test) & ( (abs(y_test - y_pred_minutes )<=15) )
    accuracy = np.mean(correct)
    coverage = np.mean(y_pred_minutes+14 >= y_test)
    print(f"Tolerance ±{tolerance} min - Accuracy: {accuracy:.2%}, Coverage: {coverage:.2%}")

    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 10 belangrijkste features:")
    print(feature_importance.head(10))

    return model, X.columns.tolist(), y_test, y_pred


In [49]:
model, feature_names, test, predict = train_decision_tree(data)

Verbeterd Decision Tree model:
R² = -0.468, RMSE = 60.28
Aantal features: 13
Tolerance ±15 min - Accuracy: 41.94%, Coverage: 69.46%

Top 10 belangrijkste features:
                       feature  importance
0           stm_progfh_in_duur    0.811326
3             stm_aanntpl_tijd    0.064615
5   tijd_verschil_meld_aanntpl    0.055352
4               stm_prioriteit    0.046549
9                stm_oorz_code    0.006118
12       stm_techn_mld_encoded    0.005572
2             stm_sap_meldtijd    0.005111
11         stm_contractgeb_mld    0.003036
7                  uur_van_dag    0.001356
10                 stm_geo_mld    0.000964


Het decision tree model blijkt een lagere RMSE en Hogere Accuracy te hebben.<br>
Maar het accuracy is hoog komt door dat bij bereken van coverage neemt we hier de grooste waarde per klassen neemt.<br>
Veder is tezien dat Coverage 10% lager is dan baselien, wat super slecht is.

### Random Forest

In [50]:
def train_random_forest(data):
    """Train verbeterd Decision Tree model met hyperparameter tuning"""
    data = data.copy()

    # Alle feature categorieën
    numeric_cols = ['stm_progfh_in_duur', 'stm_sap_melddatum', 'stm_sap_meldtijd',
                   'stm_aanntpl_tijd', 'stm_prioriteit']

    # Feature engineering: voeg afgeleide features toe
    data['tijd_verschil_meld_aanntpl'] = data['stm_aanntpl_tijd'] - data['stm_sap_meldtijd']

    # Voeg tijdsgerelateerde features toe
    data['is_werkdag'] = (data['stm_sap_melddatum'] % 7 < 5).astype(int)
    data['uur_van_dag'] = (data['stm_sap_meldtijd'] // 60).astype(int)
    data['is_spitsuur'] = ((data['uur_van_dag'] >= 7) & (data['uur_van_dag'] <= 9) |
                          (data['uur_van_dag'] >= 17) & (data['uur_van_dag'] <= 19)).astype(int)

    # Update numeric columns met nieuwe features
    numeric_cols.extend(['tijd_verschil_meld_aanntpl', 'is_werkdag',
                        'uur_van_dag', 'is_spitsuur','stm_oorz_code', 'stm_geo_mld', 'stm_contractgeb_mld','stm_techn_mld_encoded'])

    # Prepare features
    X_numeric = data[numeric_cols]
    


    # One-hot encode categorische features met frequentie filtering
    X_categorical = pd.DataFrame()

    # Combineer alle features
    X = pd.concat([X_numeric, X_categorical], axis=1)
    y = data[['targetherstel', 'herstel_klasse']]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    y_train = y_train['herstel_klasse']
    y_test = y_test['targetherstel']

    # Verbeterd Random Forest model
    model = RandomForestRegressor(
        n_estimators=500,       # Meer bomen voor betere prestaties
        random_state=42,
        max_depth=25,           # Dieper voor meer complexiteit
        min_samples_split=10,   # Lager voor meer flexibiliteit
        min_samples_leaf=5,     # Lager voor meer flexibiliteit
        max_features=0.6,       # Optimale feature subset
        bootstrap=True,
        n_jobs=-1,
        oob_score=True,         # Out-of-bag score voor extra evaluatie
        max_samples=0.8,        # Sample subsets voor diversiteit
        min_impurity_decrease=0.0005  # Lichte regularisatie
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_minutes = y_pred * 15

    # Evaluatie
    r2 = r2_score(y_pred_minutes,y_test)
    rmse = np.sqrt(mean_squared_error(y_pred_minutes,y_test))

    print("Verbeterd Random Forest model:")
    print(f"R² = {r2:.3f}, RMSE = {rmse:.2f}")
    print(f"Aantal features: {X.shape[1]}")

    tolerance =15
    correct = (y_pred_minutes +14 > y_test) & (abs(y_test - y_pred_minutes )<=15)
    accuracy = np.mean(correct)
    coverage = np.mean(y_pred_minutes+14 >= y_test)
    print(f"Tolerance ±{tolerance} min - Accuracy: {accuracy:.2%}, Coverage: {coverage:.2%}")

    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 10 belangrijkste features:")
    print(feature_importance.head(10))

    return model, X.columns.tolist(), y_test, y_pred


In [51]:
rf_model, feature_names, test, predict = train_random_forest(data)

Verbeterd Decision Tree model:
R² = -0.003, RMSE = 52.27
Aantal features: 13
Tolerance ±15 min - Accuracy: 45.97%, Coverage: 70.86%

Top 10 belangrijkste features:
                       feature  importance
0           stm_progfh_in_duur    0.571375
5   tijd_verschil_meld_aanntpl    0.082831
3             stm_aanntpl_tijd    0.072484
2             stm_sap_meldtijd    0.046716
1            stm_sap_melddatum    0.043460
10                 stm_geo_mld    0.042898
9                stm_oorz_code    0.040686
11         stm_contractgeb_mld    0.035543
12       stm_techn_mld_encoded    0.023965
4               stm_prioriteit    0.019684


In [52]:
joblib.dump(rf_model, 'RandomForest.joblib')

['RandomForest.joblib']

# sortelijk data

In [54]:
def found_data(data,stm_oorz_code,stm_geo_mld,stm_techn_mld,stm_prioriteit):
    data = data.copy()
    if stm_oorz_code:
        data = data[data['stm_oorz_code']==stm_oorz_code]
    if stm_geo_mld:
        data = data[data['stm_geo_mld']==stm_geo_mld]
    if stm_techn_mld:
        data = data[data['stm_techn_mld']==stm_techn_mld]
    if stm_prioriteit:
        data = data[data['stm_prioriteit']==stm_prioriteit]
    return data.head(10)

In [ ]:
found_data(data, stm_oorz_code=215, stm_geo_mld=120, stm_techn_mld='S', stm_prioriteit=None)